## Deploying the model as a service 

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#!pip install wandb

In [3]:
#!pip install bentoml

#### Notes:

- Ensure that you have the same version of sklearn as the version on which the pickle file was created.

- Check the sklearn version of the Google Colab platform
- Check the sklearn version here as below

Otherwise, update the sklearn version using

<code> pip install -U scikit-learn==1.0.2 </code>

Assuming, 1.0.2 is the version sklearn using which the pickle file was created. Change the version number accordingly after checking the version on Google Colab.

In [4]:
import sklearn

sklearn.__version__

'1.0.2'

In [5]:
import os
import wandb
import pandas as pd
import numpy as np

## Download the model from Weights and Biases

The model is stored under the following information

- **Project name :** 'mlops_usedcar'
- **Path to the model:** 'awesomestats/mlops_usedcar/Linear_Model_UsedCar:v2'

**Note**:

- Replace the key with your key of weights&biases (wandb)


In [6]:
import os
os.environ["WANDB_API_KEY"] = "dc797f600c763f09dc0ffc8637c8bcdf5bc1294b"

In [7]:
run = wandb.init(project='mlops_usedcar')

wandb: Currently logged in as: awesomestats. Use `wandb login --relogin` to force relogin


In [21]:
artifact = run.use_artifact('awesomestats/mlops_usedcar/Linear_Model_UsedCar:v2', type='model')
artifact_dir = artifact.download()

SyntaxError: EOL while scanning string literal (<ipython-input-21-c6d7f96897d1>, line 3)

In [23]:
f"the artifact is downloaded and stored at {artifact_dir}"

'the artifact is downloaded and stored at ./artifacts/Linear_Model_UsedCar:v2'

In [9]:
!ls -al  ./artifacts/Linear_Model_UsedCar:v2

total 24
drwxr-xr-x  3 manaranjan  staff    96 Nov 16 15:10 .
drwxr-xr-x  3 manaranjan  staff    96 Nov 16 15:10 ..
-rw-r--r--  1 manaranjan  staff  8655 Nov 16 15:10 cars.pkl


In [10]:
import joblib

In [11]:
joblib.__version__

'1.2.0'

### Load the Pickel file into Memory

In [12]:
from joblib import load

final_model = load('./artifacts/Linear_Model_UsedCar:v2/cars.pkl')

## Creating the BentoML Model Service

In [13]:
import bentoml
from bentoml.io import JSON

In [14]:
bento_model = bentoml.sklearn.save_model( "usedcar_prediction_model", 
                                         final_model,
                                         signatures={
                                            "predict": {"batchable": True, 
                                                        "batch_dim": 0},
                                        })

In [19]:
%%writefile service.py

import bentoml
import pandas as pd
import numpy as np
from bentoml.io import JSON

bento_model = bentoml.sklearn.get("usedcar_prediction_model:latest")

model_runner = bento_model.to_runner()

svc = bentoml.Service("usedcar_price_predictor", 
                      runners=[model_runner])


@svc.api(input=JSON(), output=JSON())
async def predict(input_doc: str):
        
    df = pd.DataFrame(input_doc, index = [0])
    
    num_features = ['KM_Driven', 'age', 'power_new',
                    'Seats','mileage_new', 'engine_new']    

    df[num_features] = df[num_features].apply(pd.to_numeric, 
                                              errors='coerce')
    
    print(df.info())
    
    print(df.head(1))
    
    predictions = await model_runner.predict.async_run(df)
    
    return {"ExpectedSalePrice": np.round(predictions[0], 2)}

Overwriting service.py


## Serving the model as service

### Starting the service

Start the prediction service on port 5000.

In [20]:
!bentoml serve service.py:svc --port=5000

2022-11-19T10:16:09+0530 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "service.py:svc" can be accessed at http://localhost:5000/metrics.
2022-11-19T10:16:11+0530 [INFO] [cli] Starting development HTTP BentoServer from "service.py:svc" listening on http://0.0.0.0:5000 (Press CTRL+C to quit)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   KM_Driven     1 non-null      float64
 1   Fuel_Type     1 non-null      object 
 2   age           1 non-null      float64
 3   Transmission  1 non-null      object 
 4   Owner_Type    1 non-null      object 
 5   Seats         1 non-null      float64
 6   make          1 non-null      object 
 7   mileage_new   1 non-null      float64
 8   engine_new    1 non-null      float64
 9   model         1 non-null      object 
 10  power_new     1 non-null      float64
 11  Location      1 non-null      o